In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('reviews.csv')
df.head()

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149


In [5]:
# clean the data
df['sentiment'] = df['score'].apply(lambda x: 1 if x >= 6 else 0)

texts = df['text'].values
labels = df['sentiment'].values

# labels

In [19]:
##toakenization
max_features = 10000
max_len = 500

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [20]:
import pickle
with open('mal_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [7]:
# Pad sequences to ensure uniform length
X = pad_sequences(sequences, maxlen=max_len)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


In [8]:
# Build the Simple RNN model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))




In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Early stopping callback
earlystopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping]
)

Epoch 1/10
3843/3843 [==============================] - 196s 51ms/step - loss: 0.2556 - accuracy: 0.8931 - val_loss: 0.2434 - val_accuracy: 0.8987
Epoch 2/10
3843/3843 [==============================] - 168s 44ms/step - loss: 1423.1906 - accuracy: 0.8992 - val_loss: 0.2498 - val_accuracy: 0.9003
Epoch 3/10
3843/3843 [==============================] - 152s 39ms/step - loss: nan - accuracy: 0.5436 - val_loss: nan - val_accuracy: 0.1715
Epoch 4/10
3843/3843 [==============================] - 151s 39ms/step - loss: nan - accuracy: 0.1738 - val_loss: nan - val_accuracy: 0.1715


In [16]:
model.save('anime_review_model.h5')

In [17]:
import tensorflow as tf
print("TF Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("CUDA Built:", tf.test.is_built_with_cuda())

TF Version: 2.12.0
GPU Available: []
CUDA Built: False


In [21]:
# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

1201/1201 [==============================] - 14s 12ms/step - loss: 0.2416 - accuracy: 0.9000
Test Accuracy: 0.9000


In [22]:
# Save the model
model.save('mal_reviews_sentiment_rnn.h5')


In [ ]:
# Function to predict sentiment on new reviews
def predict_sentiment(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded)
    return "Positive" if prediction > 0.5 else "Negative", float(prediction[0][0])
